## Review of Python Magic Functions

In [1]:
class C:
    def __init__(self):
        self.a = 1
        self.b = 2
    
    # This is what the @property decorator does under the hood
    def __getattr__(self, name):
        if name == "sum":
            print("Calling __getattr__ for 'sum'")
            return self.a + self.b
            
    def __getitem__(self, idx) -> int:
        if idx == 0:
            print("Calling __getitem__ for 0")
            return self.a
        elif idx == 1:
            print("Calling __getitem__ for 1")
            return self.b
        elif idx == "sum":
            print("Calling __getitem__ for 'sum'")
            return self.a + self.b

In [2]:
c = C()
print(c[0])
print(c[1])
print(c["sum"])
print(c.sum)

Calling __getitem__ for 0
1
Calling __getitem__ for 1
2
Calling __getitem__ for 'sum'
3
Calling __getattr__ for 'sum'
3


Hooks are a type of python magic that makes it (relatively) easy to link modules together, for example in the form of a sequential module. This allows us to pass multiple modules (as `*args`) into the constructor of `MySequential` and then register each fo those modules with name corresponding to the index of that module by invoking the `add_module` function of the `nn.Module` class (which is the general `Module` class of the NeuralNet library in PyTorch).

In [3]:
import torch
from torch import nn

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

Suppose we call the following constructor:

In [4]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

This goes through each of the `*args` with an inumerate:
```python
0, nn.LazyLinear(256)
1, nn.ReLU()
2, nn.LazyLinear(10)
```
and invokes the `add_module` method on `str(idx), module`, i.e. calls
```python
net.add_module("1", nn.LazyLinear(256))
net.add_module("2", nn.ReLU())
net.add_module("3", nn.LazyLinear(10))
```

In [5]:
net_constructor = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net_constructor_2 = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10, bias = False))

net_direct = nn.Sequential()
net_direct.add_module("0", nn.LazyLinear(256))
net_direct.add_module("1", nn.ReLU())
net_direct.add_module("2", nn.LazyLinear(10))

One thing the `add_module` function does is register the corresponding modules as an attribute:

In [6]:
print("\n\nnet_constructor")
print("__getattr__")
print("\t", net_constructor.__getattr__("0"))
print("\t", net_constructor.__getattr__("1"))
print("\t", net_constructor.__getattr__("2"))

print("\n__getitem__")
print("\t", net_constructor[0])
print("\t", net_constructor[1])
print("\t", net_constructor[2])

print("\nIterator")
for module_ in net_constructor:
    print("\t", module_)

try:
    print(net_constructor["1"])
except TypeError as e:
    print("\tCan't access the __getitem__ method of the module directly via the string.")
    print("\t\tWe get the error:", e)

print("\n\nnet_constructor_2")
print("\t", net_constructor_2.__getattr__("0"))
print("\t", net_constructor_2.__getattr__("1"))
print("\t", net_constructor_2.__getattr__("2"))

print("\n\nnet_direct")
print("\t", net_direct.__getattr__("0"))
print("\t", net_direct.__getattr__("1"))
print("\t", net_direct.__getattr__("2"))



net_constructor
__getattr__
	 LazyLinear(in_features=0, out_features=256, bias=True)
	 ReLU()
	 LazyLinear(in_features=0, out_features=10, bias=True)

__getitem__
	 LazyLinear(in_features=0, out_features=256, bias=True)
	 ReLU()
	 LazyLinear(in_features=0, out_features=10, bias=True)

Iterator
	 LazyLinear(in_features=0, out_features=256, bias=True)
	 ReLU()
	 LazyLinear(in_features=0, out_features=10, bias=True)
	Can't access the __getitem__ method of the module directly via the string.
		We get the error: 'str' object cannot be interpreted as an integer


net_constructor_2
	 LazyLinear(in_features=0, out_features=256, bias=True)
	 ReLU()
	 LazyLinear(in_features=0, out_features=10, bias=False)


net_direct
	 LazyLinear(in_features=0, out_features=256, bias=True)
	 ReLU()
	 LazyLinear(in_features=0, out_features=10, bias=True)


In [7]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
assert net(X).shape == nn.LazyLinear(10)(nn.ReLU()(nn.LazyLinear(256)(X))).shape